## Descrição do problema (k-centros)
Dado um conjunto de $n$ pontos e um parâmetro $k$, deseja-se encontrar $k$ centros de forma que os pontos estejam o mais próximo possível desses centros (cada ponto está localizado a uma distância máxima $r$ de um dos centros).

**Entrada:** Conjunto de pontos $S = {s_1, s_2, ..., s_n}$; Função(métrica) de distância $\text{dist}: S \times S \rightarrow \mathbb{R}^+$; Inteiro (número de centros) $k$.

**Saída:** Conjunto de centros $C = {c_1, c_2, ..., c_k}$

**Objetivo:** minimizar o $r$ máximo dos clusters, $r(C) = \max \{ \text{dist}(s_i, C) \}$ onde $\text{dist}(s_i, C) = \min \{ \text{dist}(s_i, c_j) \}$


In [2]:
# bibliotecas

import numpy as np
from ucimlrepo import fetch_ucirepo
import pandas as pd

In [11]:
# dataset (Wine)

wine = fetch_ucirepo(id=109) 
  
# dados (datasets tipo pandas)
X = wine.data.features # features de cada vinho (178 x 13), tipo contínuo/int
y = wine.data.targets  # classes dos vinhos (178 x 1), existem 3 classes distintas

samples = X.shape[0] 

### ***Métricas:*** Distância de Minkowski
A distância entre dois pontos $x$ e $y$ em $\mathbb{R}^n$ é dada por:

$$
\text{dist}(x, y) = \left( \sum_{i=1}^{n} |x_i - y_i|^p \right)^{\frac{1}{p}}
$$

In [13]:
def minkowski_dist(x,y,p):
    # x, y -> tuplas
    # p -> int
    d = 0
    for i in range(len(x)):
        d+=pow(abs(x[i]-y[i]), p)
    
    return pow(d,1/p)

### Matrizes de distância entre os pontos 
$p=1$: Distância de Manhattan

In [16]:
manhattan_d = np.zeros((samples, samples))
X_arr = X.to_numpy() if hasattr(X, 'to_numpy') else np.array(X) # convertendo o dataset pra numpy (evitar erro de indexação)

for i in range(samples):
    for j in range(i + 1, samples):
        dist = minkowski_dist(X_arr[i], X_arr[j], p=1)
        manhattan_d[i, j] = dist
        manhattan_d[j, i] = dist  # simetria
        
print(manhattan_d)

[[  0.    51.06 152.48 ... 257.14 249.41 558.28]
 [ 51.06   0.   148.3  ... 257.98 250.55 521.18]
 [152.48 148.3    0.   ... 384.76 376.83 649.14]
 ...
 [257.14 257.98 384.76 ...   0.     8.15 306.74]
 [249.41 250.55 376.83 ...   8.15   0.   312.09]
 [558.28 521.18 649.14 ... 306.74 312.09   0.  ]]


$p=2$: Distância Euclidiana

In [17]:
euclid_d = np.zeros((samples, samples))

for i in range(samples):
    for j in range(i + 1, samples):
        dist = minkowski_dist(X_arr[i], X_arr[j], p=2)
        euclid_d[i, j] = dist
        euclid_d[j, i] = dist  # simetria
        
print(euclid_d)

[[  0.          31.26501239 122.83115403 ... 230.24002302 225.21518399
  506.05936766]
 [ 31.26501239   0.         135.22469301 ... 216.22123207 211.21353863
  490.23526821]
 [122.83115403 135.22469301   0.         ... 350.57118792 345.56265177
  625.07017782]
 ...
 [230.24002302 216.22123207 350.57118792 ...   0.           5.35888981
  276.08601522]
 [225.21518399 211.21353863 345.56265177 ...   5.35888981   0.
  281.06899242]
 [506.05936766 490.23526821 625.07017782 ... 276.08601522 281.06899242
    0.        ]]


In [2]:
# Entrada: número N de samples, número k de centros, matriz dist de distâncias
# Saída: raio r, seguido de uma lista com os índices dos k centros escolhidos
# A escolha do primeiro centro é aleatória.
import random
import math
def alg_maiores_distancias(N, k, dist):
    ans = []
    ans.append(random.randrange(N))
    for i in range(k-1):
        best = -math.inf
        center = None
        for p in range(N):
            val = math.inf
            for q in ans:
                val = min(val, dist[p][q])
            if val > best:
                center = p
                best = val
        ans.append(center)

    r = -math.inf
    for p in range(N):
        best = math.inf
        for q in ans:
            best = min(best, dist(L[p], L[q]))
        r = max(r, best)

    return r, ans 


In [ ]:
alg_maiores_distancias(samples, 2, euclid